# conseption et realisation d'un dashbord et d'un modele de detection de fraude des données de la Direction des Grandes Entreprise

## Preparation des données

### Importation des modules 

In [33]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import sklearn
print(sklearn.__version__)
from platform import python_version
print(python_version())

1.2.2
3.11.7


### parametrages des modules 

In [2]:
sns.set_theme()
#pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

### selection, affichage et filtrage des fichiers

In [3]:
TVA = pd.read_excel('VraiTVA.xlsx')
#TVA

In [4]:
ERA = pd.read_excel('VraiERA.xlsx')
#ERA

In [5]:
jointure = pd.merge(ERA, TVA, on='BP', how='inner')
#jointure = jointure[['BP', 'Wilaya', 'Code CNRC', 'Code ONS', 'Chiffre d’affaire (C.A)', 'Total TVA anuelle']]  

In [6]:
jointure.rename(columns={'Chiffre d’affaire (C.A)': 'ChAff'}, inplace=True)
jointure.rename(columns={'Total TVA anuelle': 'Total TVA anunelle'}, inplace=True)
#jointure.head()

### créer la colonne feature et cible


In [7]:
jointure['feature'] =  jointure['ChAff']  - jointure['Total TVA anunelle'] 

In [8]:
def estime(row):
    if row['ChAff'] == 0 or row['Total TVA anunelle'] ==0 :
        return 'sus'
    elif abs(row['feature']) > 1000:
        return 'fraude'
    else:
        return 'bon'

jointure['cible'] = jointure.apply(estime, axis=1)


In [9]:
jointure.head()

,BP,Wilaya,Code CNRC,Code ONS,ChAff,Total TVA Janvier,Total TVA Février,Total TVA Mars,Total TVA Avril,Total TVA Mai,Total TVA Juin,Total TVA Juillet,Total TVA Août,Total TVA Septembre,Total TVA Octobre,Total TVA Novembre,Total TVA Décembre,Total TVA anunelle,feature,cible
0,2000000147,DIW Alger est,607047,/,0,4.381490e+09,4.710011e+09,5.358351e+09,4.811742e+09,4.835182e+09,4.467203e+09,3.692317e+09,3.713033e+09,5.274394e+09,6.056911e+09,4.706919e+09,5.852010e+09,5.785956e+10,-5.785956e+10,sus
1,2000000200,DIW Alger est,608001,/,93115698,6.441810e+05,1.478702e+06,2.304256e+06,1.226741e+06,2.147761e+07,3.343922e+07,9.184847e+06,4.261077e+06,3.033041e+06,4.966790e+06,5.425418e+06,5.673810e+06,9.311570e+07,0.000000e+00,bon
2,2000007178,DIW Bordj Bou Arréridj,110202,/,155423274,4.872747e+06,4.275042e+06,3.918504e+06,6.082352e+06,7.774514e+06,2.706730e+07,2.240796e+07,8.738202e+06,5.113770e+06,5.666626e+06,2.643794e+07,3.306832e+07,1.554233e+08,0.000000e+00,bon
3,2000009570,DIW Alger est,405105,/,5798168362,4.232086e+08,3.965581e+08,4.111293e+08,6.288660e+08,3.132802e+08,4.100639e+08,3.332674e+08,5.615210e+08,7.133190e+08,4.950724e+08,5.964098e+08,5.272515e+08,5.809947e+09,-1.177863e+07,fraude
4,2000011509,DIW Alger est,409001,/,2062602782,1.607687e+08,2.937824e+08,3.321080e+08,1.234689e+08,1.051586e+08,2.330517e+08,1.305186e+08,1.720886e+08,1.598969e+08,1.437596e+08,7.488136e+07,1.328862e+08,2.062370e+09,2.331380e+05,fraude


## Over sample de la donnée fraude dans le data set

In [10]:
# Get the counts of each class
fraud_count = jointure['cible'].value_counts()
print(fraud_count)

# Specify the class you want to oversample, in this case, 'fraude'
target_class = 'fraude'

# Define the number of samples you want for the 'fraude' class after oversampling
# Here, you can use a multiplier to specify how many times larger the 'fraude' class should be
multiplier = 1
target_sample_count = fraud_count[target_class] * multiplier

# Calculate the additional samples you need
additional_samples = target_sample_count - fraud_count[target_class]

# Filter the 'fraude' class
fraude_data = jointure[jointure['cible'] == target_class]

# Randomly sample from the 'fraude' data
oversampled_fraude = fraude_data.sample(n=additional_samples, replace=True, random_state=42)

# Append the oversampled data to the original DataFrame
jointure_oversampled = pd.concat([jointure, oversampled_fraude])

# Shuffle the dataset to mix the rows up
jointure_oversampled = jointure_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)


cible
fraude    720
sus       340
bon       210
Name: count, dtype: int64


### Verification du nouveau data set

In [11]:
print(jointure_oversampled['cible'].value_counts())

cible
fraude    720
sus       340
bon       210
Name: count, dtype: int64


In [12]:
print(jointure_oversampled.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1270 entries, 0 to 1269
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BP                   1270 non-null   int64  
 1   Wilaya               1270 non-null   object 
 2   Code CNRC            1270 non-null   object 
 3   Code ONS             1270 non-null   object 
 4   ChAff                1270 non-null   int64  
 5   Total TVA Janvier    1270 non-null   float64
 6   Total TVA Février    1270 non-null   float64
 7   Total TVA Mars       1270 non-null   float64
 8   Total TVA Avril      1270 non-null   float64
 9   Total TVA Mai        1270 non-null   float64
 10  Total TVA Juin       1270 non-null   float64
 11  Total TVA Juillet    1270 non-null   float64
 12  Total TVA Août       1270 non-null   float64
 13  Total TVA Septembre  1270 non-null   float64
 14  Total TVA Octobre    1270 non-null   float64
 15  Total TVA Novembre   1270 non-null   f

## Encodage categoriel & Normalisation/Standardisation 

### encodage categoriel des wilaya

In [13]:
wilaya_count = jointure_oversampled['Wilaya'].value_counts()
print(wilaya_count)

Wilaya
DIW Alger centre          520
DIW Alger est             188
DIW Alger ouest           107
DIW Oran Est               71
DIW Blida                  45
DIW Ouargla                44
DIW Constantine            31
DIW Boumerdès              30
DIW Sétif                  27
DIW Béjaïa                 23
DIW Annaba                 19
DIW Sidi Bel Abbes         17
DIW Bordj Bou Arréridj     15
DIW Batna                  14
DIW Skikda                 12
DIW Tizi Ouzou             10
DIW Tipaza                  9
DIW Tlemcen                 9
DIW Bouira                  8
DIW Mostaganem              7
DIW M'Sila                  7
DIW Jijel                   6
DIW Chlef                   6
DIW Oum el-Bouaghi          5
DIW Biskra                  5
DIW Mila                    4
DIW Aïn Témouchent          3
Non affecté                 3
DIW Relizane                3
DIW Médéa                   3
DIW Mascara                 3
DIW Tiaret                  2
DIW Guelma                  2
DIW

In [14]:
# Clean and standardize the Wilaya names
jointure_oversampled['Standardized_Wilaya'] = jointure_oversampled['Wilaya'].str.replace('DIW ', '')

# Label Encoding
wilaya_mapping = {name: i + 1 for i, name in enumerate(jointure_oversampled['Standardized_Wilaya'].unique())}
jointure_oversampled['Wilaya_encoded'] = jointure_oversampled['Standardized_Wilaya'].map(wilaya_mapping)

# Display the DataFrame to check the new columns
print(jointure_oversampled[['Wilaya', 'Standardized_Wilaya', 'Wilaya_encoded']])

                Wilaya Standardized_Wilaya  Wilaya_encoded
0      DIW Alger ouest         Alger ouest               1
1     DIW Alger centre        Alger centre               2
2        DIW Alger est           Alger est               3
3     DIW Alger centre        Alger centre               2
4     DIW Alger centre        Alger centre               2
...                ...                 ...             ...
1265     DIW Alger est           Alger est               3
1266  DIW Alger centre        Alger centre               2
1267  DIW Alger centre        Alger centre               2
1268   DIW Alger ouest         Alger ouest               1
1269      DIW Relizane            Relizane              11

[1270 rows x 3 columns]


### encodage categoriel des Codes ONS et CNRC

In [15]:
# Label Encoding for 'code CNRC'
cnrc_unique = pd.unique(jointure_oversampled['Code CNRC'])
code_cnrc_mapping = {code: idx + 1 for idx, code in enumerate(cnrc_unique)}
jointure_oversampled['code CNRC_encoded'] = jointure_oversampled['Code CNRC'].map(code_cnrc_mapping)

# Label Encoding for 'code ONS'
ons_unique = pd.unique(jointure_oversampled['Code ONS'])
code_ons_mapping = {code: idx + 1 for idx, code in enumerate(ons_unique)}
jointure_oversampled['code ONS_encoded'] = jointure_oversampled['Code ONS'].map(code_ons_mapping)

# Display the DataFrame to check the new columns
print(jointure_oversampled)

              BP            Wilaya Code CNRC Code ONS        ChAff  \
0     2000048567   DIW Alger ouest    101105        /            0   
1     2000046698  DIW Alger centre         /        /   4374440594   
2     2000045180     DIW Alger est    302401        /   6371606525   
3     2000046267  DIW Alger centre         /        /   2929819984   
4     2000045730  DIW Alger centre         /        /  13795641104   
...          ...               ...       ...      ...          ...   
1265  2000046450     DIW Alger est    613203     4329            0   
1266  2000046555  DIW Alger centre    613203     4329            0   
1267  2000046633  DIW Alger centre    613125     4321     22988920   
1268  2000045829   DIW Alger ouest         /        /    113184009   
1269  2000046628      DIW Relizane    613203     4329            0   

      Total TVA Janvier  Total TVA Février  Total TVA Mars  Total TVA Avril  \
0          3.360857e+08       0.000000e+00    0.000000e+00              0.0   
1

### encodage categoriel de target ENFIN

In [16]:
target_mapping = {
    'bon': 0,      # Typically, the "normal" class can be zero
    'fraude': 1,   # Positive class can be one, especially if it's a binary classification
    'sus': 2       # Additional class can take the next integer
}

# Apply the mapping to the target column
jointure_oversampled['target_encoded'] = jointure_oversampled['cible'].map(target_mapping)

# Display the DataFrame to check the new column
print(jointure_oversampled[['cible', 'target_encoded']])

       cible  target_encoded
0        sus               2
1     fraude               1
2     fraude               1
3     fraude               1
4     fraude               1
...      ...             ...
1265     sus               2
1266     sus               2
1267  fraude               1
1268     sus               2
1269     sus               2

[1270 rows x 2 columns]


### verification du Data set :SOB: + supression des columns not needed 

In [17]:
jointure_oversampled

,BP,Wilaya,Code CNRC,Code ONS,ChAff,Total TVA Janvier,Total TVA Février,Total TVA Mars,Total TVA Avril,Total TVA Mai,Total TVA Juin,Total TVA Juillet,Total TVA Août,Total TVA Septembre,Total TVA Octobre,Total TVA Novembre,Total TVA Décembre,Total TVA anunelle,feature,cible,Standardized_Wilaya,Wilaya_encoded,code CNRC_encoded,code ONS_encoded,target_encoded
0,2000048567,DIW Alger ouest,101105,/,0,3.360857e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,77791324.0,125811892.0,957691974.0,1.170789e+08,5.675235e+07,0.000000e+00,1.671212e+09,-1.671212e+09,sus,Alger ouest,1,1,1,2
1,2000046698,DIW Alger centre,/,/,4374440594,2.784815e+09,0.000000e+00,0.000000e+00,0.0,1.238461e+09,0.0,637620686.0,164100124.0,148556827.0,1.862119e+08,0.000000e+00,3.070375e+08,5.466804e+09,-1.092363e+09,fraude,Alger centre,2,2,1,1
2,2000045180,DIW Alger est,302401,/,6371606525,5.239990e+08,5.237330e+08,6.265229e+08,462952996.0,5.420719e+08,553601535.0,503753058.0,567357515.0,570455559.0,5.699451e+08,5.455203e+08,5.420279e+08,6.531941e+09,-1.603342e+08,fraude,Alger est,3,3,1,1
3,2000046267,DIW Alger centre,/,/,2929819984,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.929453e+08,100547189.0,299271381.0,309586048.0,330413933.0,0.000000e+00,8.184679e+08,7.333412e+08,2.784573e+09,1.452469e+08,fraude,Alger centre,2,2,1,1
4,2000045730,DIW Alger centre,/,/,13795641104,1.097884e+09,1.571380e+09,1.109034e+09,599193629.0,6.429233e+08,853101155.0,847689347.0,766807075.0,781724338.0,1.283896e+09,1.290737e+09,1.535100e+09,1.237947e+10,1.416170e+09,fraude,Alger centre,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,2000046450,DIW Alger est,613203,4329,0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,sus,Alger est,3,8,2,2
1266,2000046555,DIW Alger centre,613203,4329,0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,sus,Alger centre,2,8,2,2
1267,2000046633,DIW Alger centre,613125,4321,22988920,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,2.954700e+06,2.954700e+06,2.003422e+07,fraude,Alger centre,2,25,5,1
1268,2000045829,DIW Alger ouest,/,/,113184009,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.131840e+08,sus,Alger ouest,1,2,1,2


In [18]:
print(jointure_oversampled.columns)

Index(['BP', 'Wilaya', 'Code CNRC', 'Code ONS', 'ChAff', 'Total TVA Janvier',
       'Total TVA Février', 'Total TVA Mars', 'Total TVA Avril',
       'Total TVA Mai', 'Total TVA Juin', 'Total TVA Juillet',
       'Total TVA Août', 'Total TVA Septembre', 'Total TVA Octobre',
       'Total TVA Novembre', 'Total TVA Décembre', 'Total TVA anunelle',
       'feature', 'cible', 'Standardized_Wilaya', 'Wilaya_encoded',
       'code CNRC_encoded', 'code ONS_encoded', 'target_encoded'],
      dtype='object')


In [19]:
jointure_oversampled = jointure_oversampled.drop(columns = ['Wilaya', 'Code CNRC', 'Code ONS','cible'])

In [20]:
jointure_oversampled = jointure_oversampled.drop(columns = ['Standardized_Wilaya'])

In [21]:
jointure_oversampled

,BP,ChAff,Total TVA Janvier,Total TVA Février,Total TVA Mars,Total TVA Avril,Total TVA Mai,Total TVA Juin,Total TVA Juillet,Total TVA Août,Total TVA Septembre,Total TVA Octobre,Total TVA Novembre,Total TVA Décembre,Total TVA anunelle,feature,Wilaya_encoded,code CNRC_encoded,code ONS_encoded,target_encoded
0,2000048567,0,3.360857e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,77791324.0,125811892.0,957691974.0,1.170789e+08,5.675235e+07,0.000000e+00,1.671212e+09,-1.671212e+09,1,1,1,2
1,2000046698,4374440594,2.784815e+09,0.000000e+00,0.000000e+00,0.0,1.238461e+09,0.0,637620686.0,164100124.0,148556827.0,1.862119e+08,0.000000e+00,3.070375e+08,5.466804e+09,-1.092363e+09,2,2,1,1
2,2000045180,6371606525,5.239990e+08,5.237330e+08,6.265229e+08,462952996.0,5.420719e+08,553601535.0,503753058.0,567357515.0,570455559.0,5.699451e+08,5.455203e+08,5.420279e+08,6.531941e+09,-1.603342e+08,3,3,1,1
3,2000046267,2929819984,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.929453e+08,100547189.0,299271381.0,309586048.0,330413933.0,0.000000e+00,8.184679e+08,7.333412e+08,2.784573e+09,1.452469e+08,2,2,1,1
4,2000045730,13795641104,1.097884e+09,1.571380e+09,1.109034e+09,599193629.0,6.429233e+08,853101155.0,847689347.0,766807075.0,781724338.0,1.283896e+09,1.290737e+09,1.535100e+09,1.237947e+10,1.416170e+09,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,2000046450,0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3,8,2,2
1266,2000046555,0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2,8,2,2
1267,2000046633,22988920,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,2.954700e+06,2.954700e+06,2.003422e+07,2,25,5,1
1268,2000045829,113184009,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.131840e+08,1,2,1,2


### normalisation et standardisation 
 vu que nous allons faire a random forest and/or a regression tree, il serait preferable de standardiser les données 

In [22]:
jointure_oversampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1270 entries, 0 to 1269
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BP                   1270 non-null   int64  
 1   ChAff                1270 non-null   int64  
 2   Total TVA Janvier    1270 non-null   float64
 3   Total TVA Février    1270 non-null   float64
 4   Total TVA Mars       1270 non-null   float64
 5   Total TVA Avril      1270 non-null   float64
 6   Total TVA Mai        1270 non-null   float64
 7   Total TVA Juin       1270 non-null   float64
 8   Total TVA Juillet    1270 non-null   float64
 9   Total TVA Août       1270 non-null   float64
 10  Total TVA Septembre  1270 non-null   float64
 11  Total TVA Octobre    1270 non-null   float64
 12  Total TVA Novembre   1270 non-null   float64
 13  Total TVA Décembre   1270 non-null   float64
 14  Total TVA anunelle   1270 non-null   float64
 15  feature              1270 non-null   f

In [23]:
columns_to_standardize = [
    'ChAff',
    'Total TVA Janvier', 'Total TVA Février', 'Total TVA Mars', 'Total TVA Avril',
    'Total TVA Mai', 'Total TVA Juin', 'Total TVA Juillet', 'Total TVA Août',
    'Total TVA Septembre', 'Total TVA Octobre', 'Total TVA Novembre', 'Total TVA Décembre',
    'Total TVA anunelle'
]

# Apply standardization
for column in columns_to_standardize:
    jointure_oversampled[column] = (jointure_oversampled[column] - jointure_oversampled[column].mean()) / jointure_oversampled[column].std()

# Now your columns are standardized
print(jointure_oversampled[columns_to_standardize].head())

      ChAff  Total TVA Janvier  Total TVA Février  Total TVA Mars  \
0 -0.064956          -0.078942          -0.206763       -0.218806   
1 -0.047351           0.730177          -0.206763       -0.218806   
2 -0.039313          -0.016851           0.002844        0.008082   
3 -0.053165          -0.189993          -0.206763       -0.218806   
4 -0.009434           0.172774           0.422130        0.182818   

   Total TVA Avril  Total TVA Mai  Total TVA Juin  Total TVA Juillet  \
0        -0.196989      -0.200689       -0.209758          -0.169582   
1        -0.196989       0.215054       -0.209758           0.042430   
2        -0.031979      -0.018719       -0.009858          -0.008267   
3        -0.196989      -0.135918       -0.173451          -0.085706   
4         0.016582       0.015136        0.098288           0.121985   

   Total TVA Août  Total TVA Septembre  Total TVA Octobre  Total TVA Novembre  \
0       -0.161003             0.138830          -0.176747           -0.

## MODEL SKLEARN KNN 

In [24]:
jointure_oversampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1270 entries, 0 to 1269
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   BP                   1270 non-null   int64  
 1   ChAff                1270 non-null   float64
 2   Total TVA Janvier    1270 non-null   float64
 3   Total TVA Février    1270 non-null   float64
 4   Total TVA Mars       1270 non-null   float64
 5   Total TVA Avril      1270 non-null   float64
 6   Total TVA Mai        1270 non-null   float64
 7   Total TVA Juin       1270 non-null   float64
 8   Total TVA Juillet    1270 non-null   float64
 9   Total TVA Août       1270 non-null   float64
 10  Total TVA Septembre  1270 non-null   float64
 11  Total TVA Octobre    1270 non-null   float64
 12  Total TVA Novembre   1270 non-null   float64
 13  Total TVA Décembre   1270 non-null   float64
 14  Total TVA anunelle   1270 non-null   float64
 15  feature              1270 non-null   f

In [25]:
# Select your features and target variable
X = jointure_oversampled[['feature']]
y = jointure_oversampled['target_encoded'] 

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the k-NN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Make predictions on the test set
predictions = knn.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, predictions))

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.74      0.85        31
           1       0.85      0.91      0.88       152
           2       0.68      0.65      0.66        71

    accuracy                           0.81       254
   macro avg       0.84      0.77      0.80       254
weighted avg       0.82      0.81      0.81       254

Confusion Matrix:
[[ 23   0   8]
 [  0 138  14]
 [  0  25  46]]


In [26]:
import joblib
joblib.dump(knn, 'MRknn_model.joblib')


['MRknn_model.joblib']

In [27]:
pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: C:\Users\Administrateur\Downloads\anaconda\Lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: imbalanced-learn
Note: you may need to restart the kernel to use updated packages.
